# Sensitive Model Search

by adjusting `o_random_seed` and `parameter_generation_seed`

## Init

In [12]:
import os

path = os.getcwd()
# find the string 'project' in the path, return index
index_project = path.find('project')
# slice the path from the index of 'project' to the end
project_path = path[:index_project+7]
# set the working directory
os.chdir(project_path+'/src')
print(f'Project path set to: {os.getcwd()}')

Project path set to: /mnt/userdata2/dawsonlan/new-peak-project/src


In [13]:
from dotenv import dotenv_values
config = dotenv_values(".env")
print(config["DATA_PATH"])

/home/dawsonlan/new-peak-project/src/experiments


In [14]:
from models.ModelBuilder import ModelBuilder
from models.Reaction import Reaction
from models.ReactionArchtype import ReactionArchtype
from models.ArchtypeCollections import *
from models.Utils import *

import matplotlib.pyplot as plt
import seaborn as sns
import roadrunner
import numpy as np
import pandas as pd

# import scikit-learn
from sklearn.linear_model import LinearRegression
# tree models and support vector machines
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# import pearson correlation
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from copy import deepcopy

## Analysis

In [39]:
import os 

### parameters 
notebook_name = 'exp4_drug_model_search'
sub_id = '2'

## Model parameters
no_observable_species = 15
no_feedback_regulations = 10
specie_value_range = (5, 5000)
param_range = (0.05, 20)
param_multiplier_range = (0.5, 1.5)
model_name = 'sensitive_model_search'

drug_name = 'D0'
drug_conc = 5000
drug_time = 500

## Sensitivity analysis parameters 
o_random_seeds = list(range(1,10))
parameter_random_seeds = list(range(1, 10))
species_perturbation_range = np.arange(1, 5000, 500)

## Simulation parameters 
simulation_time = 1000 
simulation_step = 100

## General parameters
parallelise = True
save_figures = True 
experiment_id = notebook_name + '_' + sub_id
experiment_folder = config['DATA_PATH'] + '/' + experiment_id + '/'
if not os.path.exists(experiment_folder):
    os.makedirs(experiment_folder)
    
print(experiment_folder)

/home/dawsonlan/new-peak-project/src/experiments/exp4_drug_model_search_2/


In [40]:
## Helper functions
import warnings

from models.SensitivityAnalysis import sensitivity_analysis, get_sensitivity_score, extract_states_from_results

In [ ]:
results = []
# use joblib to parallelise the code
from joblib import Parallel, delayed
from models.Solver.RoadrunnerSolver import RoadrunnerSolver
from models.DrugModelSpecification import DrugModelSpecification, Drug

def run_sensitivity_analysis(o_random_seed, parameter_random_seed, verbose=0):
    # Generate the model 
    model_spec = DrugModelSpecification()
    model_spec.generate_specifications(o_random_seed, no_observable_species, no_feedback_regulations, verbose=0)
    drug_0 = Drug(drug_name, drug_time, drug_conc)
    np.random.seed(o_random_seed)
    # add random 'up' and 'down' regulations to the drug
    regulation_dir = []
    for i, s in enumerate(model_spec.A_species):
        regulation_dir.append(np.random.choice(['up', 'down']))
        drug_0.add_regulation(s, 'up')
    model_spec.add_drug(drug_0)
    G0 = model_spec.generate_network(model_name, specie_value_range, param_range, param_multiplier_range, random_seed=parameter_random_seed, verbose=0)
    solver = RoadrunnerSolver()
    solver.compile(G0.get_sbml_model())
    all_states = []
    for i in range(no_observable_species):
        all_states.append('A'+str(i))
    for i in range(no_observable_species):
        all_states.append('B'+str(i))
                
    all_init_species_results = []
    for init_species in all_states: 
        all_results = sensitivity_analysis(G0, solver, init_species, species_perturbation_range, simulation_time, simulation_step)
        all_init_species_results.append(all_results)

    # extract the last time point of Cp for each init species
    Cp_final_states = []
    for init_species in all_init_species_results: 
        Cp_final_states.append(extract_states_from_results(init_species, 'Cp', -1))
        
    state_sensitivity = get_sensitivity_score(Cp_final_states)
    # get a balance of sensitiv
    mean = np.mean(state_sensitivity)
    std_dev = np.std(state_sensitivity)
    cv = std_dev / mean
    # get the mean of the sensitivity score
    sens_score = np.mean(state_sensitivity)
    median = np.median(state_sensitivity)
    if verbose: 
        print(f'Random seed: {o_random_seed}, Parameter random seed: {parameter_random_seed}, Mean: {sens_score}, CV: {cv}, Median: {median}')
        if verbose > 1:
            print('State sensitivity as a function of init species:')
            for i, init_species in enumerate(all_init_species_results): 
                print(f'Init species: {init_species}, Sensitivity score: {state_sensitivity[i]}')
    return [o_random_seed, parameter_random_seed, cv, sens_score, median]





In [42]:
res = run_sensitivity_analysis(9, 1, verbose=1)

Random seed: 9, Parameter random seed: 1, Mean: 5.321714045895817, CV: 3.631693803713879, Median: 0.19123714940673842
[np.float64(54.55916367423416), np.float64(0.006415957491914703), np.float64(0.4621862115959483), np.float64(0.08919464683404055), np.float64(0.21586892783498435), np.float64(0.022648079253090536), np.float64(0.3218987592513747), np.float64(0.06386523055098792), np.float64(0.6165760476840418), np.float64(0.1051361012341947), np.float64(0.026228487494004415), np.float64(0.01020371454984259), np.float64(0.001211948559912912), np.float64(0.12472360767107027), np.float64(0.0031512214098192715), np.float64(95.23875104814081), np.float64(0.25518850801165227), np.float64(0.5737508830748936), np.float64(0.8140131091603182), np.float64(1.2587283180201907), np.float64(0.2295427392082705), np.float64(1.011242100403781), np.float64(1.1270543079478075), np.float64(1.1280630609884543), np.float64(0.012008392204210683), np.float64(0.16660537097849248), np.float64(0.07058488367311844),

In [43]:
from joblib import cpu_count
print(f'Joblib running on CPU cores: {cpu_count()}')

results = Parallel(n_jobs=-1)(delayed(run_sensitivity_analysis)(o_random_seed, parameter_random_seed) for o_random_seed in o_random_seeds for parameter_random_seed in parameter_random_seeds)
# convert to pandas dataframe
df = pd.DataFrame(results, columns=['o_random_seed', 'parameter_random_seed', 'cv', 'mean', 'median'])

# sort the dataframe by sensitivity score
df = df.sort_values(by='median', ascending=False)
df

Joblib running on CPU cores: 12


,o_random_seed,parameter_random_seed,cv,mean,median
80,9,9,2.179187,4.995138,2.364699
76,9,5,2.516145,7.250063,2.287098
77,9,6,2.519659,5.399918,1.937970
51,6,7,2.016463,5.378363,1.746281
54,7,1,1.374436,1.721651,1.176730
...,...,...,...,...,...
6,1,7,1.601287,0.055525,0.000783
25,3,8,2.140776,0.000835,0.000259
4,1,5,1.746829,0.013776,0.000236
1,1,2,1.344134,0.028852,0.000058


In [ ]:
# import differential evolution 

In [44]:
# save df to a pickle file
df.to_pickle(experiment_folder + 'sensitivity_analysis_results.pkl')
print(f'Saved results to {experiment_folder + "sensitivity_analysis_results.pkl"}')

Saved results to /home/dawsonlan/new-peak-project/src/experiments/exp4_drug_model_search_2/sensitivity_analysis_results.pkl
